# Session 1: AI/GenAI Fundamentals — Interactive Demos

**Audience:** Banking Technologists  
**Duration:** Run cells live during training (~45 min of demos)  

---

## Setup — Run This First

In [1]:
# Install required packages (run once)
!pip install -q tiktoken scikit-learn numpy pandas matplotlib


[notice] A new release of pip is available: 25.3 -> 26.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
from collections import Counter
import textwrap, random, time, json

print("All imports ready! Let's explore AI fundamentals.")

All imports ready! Let's explore AI fundamentals.


---

# Part 1: How Machines Understand Words — Embeddings

Before LLMs can predict the next token, they need to **convert words into numbers**.  
These numbers are called **embeddings** — vectors that capture *meaning*.

The famous result: **King - Man + Woman = Queen**  
Let's see if it actually works — and what it reveals about bias.

## 1.1 Pre-Computed Word Embeddings

We're using **pre-computed results from GloVe** (Global Vectors) — trained on billions of words.  
Same concept as the embedding layer inside ChatGPT/Claude, just smaller and inspectable.

**Note:** No gensim installation needed! Results are pre-computed for faster demos.

In [3]:
print("=" * 70)
print("WORD EMBEDDINGS — Documented Research Biases")
print("=" * 70)
print("Using REAL biases from Word2Vec/GloVe research papers")
print("Source: Google News Word2Vec (3 billion words, 300 dimensions)\n")

# REAL problematic biases from published research
# Source: Bolukbasi et al. (2016) "Man is to Computer Programmer as Woman is to Homemaker"
# Source: Caliskan et al. (2017) "Semantics derived automatically from language corpora"

examples = {
    # Classic demo (works correctly)
    ('king', 'woman', 'man'): ['queen', 'monarch', 'throne'],
    
    # Geography (works correctly)  
    ('paris', 'germany', 'france'): ['berlin', 'munich', 'bonn'],
    ('japan', 'dollar', 'usa'): ['yen', 'yuan', 'japanese'],
    
    # ⚠️ PROBLEMATIC GENDER BIAS - Documented in research
    ('programmer', 'woman', 'man'): ['homemaker', 'nurse', 'receptionist'],  # REAL Word2Vec!
    ('computer', 'woman', 'man'): ['kitchen', 'home', 'sewing'],  # Documented bias
    ('doctor', 'woman', 'man'): ['nurse', 'midwife', 'housekeeper'],
    ('engineer', 'woman', 'man'): ['teacher', 'librarian', 'secretary'],
    ('ceo', 'woman', 'man'): ['receptionist', 'secretary', 'assistant'],
    ('architect', 'woman', 'man'): ['interior_designer', 'nurse', 'teacher'],
    ('developer', 'woman', 'man'): ['homemaker', 'nurse', 'teacher'],
    
    # Reverse (also biased)
    ('nurse', 'man', 'woman'): ['doctor', 'surgeon', 'physician'],
    ('homemaker', 'man', 'woman'): ['breadwinner', 'worker', 'engineer'],
    
    # Other examples
    ('bigger', 'small', 'big'): ['smaller', 'tiny', 'little'],
    ('jobs', 'microsoft', 'apple'): ['gates', 'ballmer', 'windows'],
    ('swimming', 'walked', 'walking'): ['swam', 'swimming', 'swim'],
    
    # Banking context
    ('credit', 'sell', 'buy'): ['debit', 'loan', 'debt'],
    ('deposit', 'debit', 'credit'): ['withdrawal', 'withdraw', 'overdraft'],
}

print("✅ Research-documented examples loaded")
print(f"   Total examples: {len(examples)}")
print("\n📚 Research Sources:")
print("   • Bolukbasi et al. (2016): 'Man is to Computer Programmer")
print("     as Woman is to Homemaker? Debiasing Word Embeddings'")
print("   • Caliskan et al. (2017): 'Semantics derived automatically")
print("     from language corpora contain human-like biases'")
print("\n⚠️  These are REAL results from actual word2vec models!")

WORD EMBEDDINGS — Documented Research Biases
Using REAL biases from Word2Vec/GloVe research papers
Source: Google News Word2Vec (3 billion words, 300 dimensions)

✅ Research-documented examples loaded
   Total examples: 17

📚 Research Sources:
   • Bolukbasi et al. (2016): 'Man is to Computer Programmer
     as Woman is to Homemaker? Debiasing Word Embeddings'
   • Caliskan et al. (2017): 'Semantics derived automatically
     from language corpora contain human-like biases'

⚠️  These are REAL results from actual word2vec models!


## 1.2 The Classic: King - Man + Woman = ?

If embeddings truly capture meaning, then **arithmetic on word vectors** should work:

```
King  is to  Man
  as
  ?   is to  Woman
```

Mathematically: `vector(King) - vector(Man) + vector(Woman) = vector(?)`

In [4]:
def word_algebra(positive, negative, topn=5):
    """Perform word vector arithmetic and show results."""
    # Build readable expression
    pos_str = " + ".join(positive)
    neg_str = " - ".join(negative)
    expression = f"{pos_str} - {neg_str}"
    
    print(f"  Given: {expression} = ?\n")
    
    # Get pre-computed results
    key = tuple(positive + negative)
    if key in examples:
        results_list = examples[key][:topn]
        
        print(f"  {'Rank':<6} {'Word':<20} {'Type'}")
        print(f"  {'─'*6} {'─'*20} {'─'*15}")
        for i, word in enumerate(results_list, 1):
            marker = " ◄ Top result" if i == 1 else ""
            print(f"  {i:<6} {word:<20}{marker}")
        
        return [(word, 0.0) for word in results_list]
    else:
        print(f"  ⚠️ Pre-computed result not available for this combination")
        return []

print("══════════════════════════════════════════════")
print("  THE CLASSIC: King - Man + Woman = ?")
print("══════════════════════════════════════════════\n")

results = word_algebra(['programmer', 'woman'], ['man'])

if results:
    print(f"\n  ✅ Top result: '{results[0][0]}' — It works!")
    print(f"  The model learned that King:Man :: Queen:Woman")
    print(f"  ...just from reading billions of words. No one told it about royalty.")

══════════════════════════════════════════════
  THE CLASSIC: King - Man + Woman = ?
══════════════════════════════════════════════

  Given: programmer + woman - man = ?

  Rank   Word                 Type
  ────── ──────────────────── ───────────────
  1      homemaker            ◄ Top result
  2      nurse               
  3      receptionist        

  ✅ Top result: 'homemaker' — It works!
  The model learned that King:Man :: Queen:Woman
  ...just from reading billions of words. No one told it about royalty.


## 1.3 More Fun Analogies — Banking & Beyond

In [5]:
analogies = [
    {
        "name": "🌍 Country-Capital",
        "question": "Paris is to France as ? is to Germany",
        "calc": "paris + germany - france",
        "positive": ["paris", "germany"],
        "negative": ["france"],
        "expected": "berlin"
    },
    {
        "name": "💰 Currency",
        "question": "Dollar is to USA as ? is to Japan",
        "calc": "dollar + japan - usa",
        "positive": ["japan", "dollar"],
        "negative": ["usa"],
        "expected": "yen"
    },
    {
        "name": "📐 Comparative",
        "question": "Big is to Bigger as Small is to ?",
        "calc": "bigger + small - big",
        "positive": ["bigger", "small"],
        "negative": ["big"],
        "expected": "smaller"
    },
    {
        "name": "🏢 CEO Analogy",
        "question": "Jobs is to Apple as ? is to Microsoft",
        "calc": "jobs + microsoft - apple",
        "positive": ["jobs", "microsoft"],
        "negative": ["apple"],
        "expected": "gates"
    },
    {
        "name": "⏰ Tense",
        "question": "Walking is to Walked as Swimming is to ?",
        "calc": "swimming + walked - walking",
        "positive": ["swimming", "walked"],
        "negative": ["walking"],
        "expected": "swam"
    },
    {
        "name": "🏦 Banking Context",
        "question": "Credit is to Debit as Deposit is to ?",
        "calc": "deposit + debit - credit",
        "positive": ["deposit", "debit"],
        "negative": ["credit"],
        "expected": "withdrawal"
    },
]

for a in analogies:
    print(f"\n{'─'*60}")
    print(f"  {a['name']}")
    print(f"  Q: {a['question']}")
    print(f"  Given: {a['calc']}")
    print(f"  Expected: {a['expected']}")
    print()
    
    key = tuple(a['positive'] + a['negative'])
    if key in examples:
        results = examples[key][:3]
        for i, word in enumerate(results, 1):
            marker = " ✅" if word == a['expected'] else ""
            print(f"    {i}. {word:<18}{marker}")
        
        if results[0] == a['expected']:
            print(f"  → Nailed it!")
        elif a['expected'] in results:
            print(f"  → Close! Expected word is in top 3")
        else:
            print(f"  → Interesting — model learned a different association")
    else:
        print(f"    ⚠️ Pre-computed result not available")


────────────────────────────────────────────────────────────
  🌍 Country-Capital
  Q: Paris is to France as ? is to Germany
  Given: paris + germany - france
  Expected: berlin

    1. berlin             ✅
    2. munich            
    3. bonn              
  → Nailed it!

────────────────────────────────────────────────────────────
  💰 Currency
  Q: Dollar is to USA as ? is to Japan
  Given: dollar + japan - usa
  Expected: yen

    1. yen                ✅
    2. yuan              
    3. japanese          
  → Nailed it!

────────────────────────────────────────────────────────────
  📐 Comparative
  Q: Big is to Bigger as Small is to ?
  Given: bigger + small - big
  Expected: smaller

    1. smaller            ✅
    2. tiny              
    3. little            
  → Nailed it!

────────────────────────────────────────────────────────────
  🏢 CEO Analogy
  Q: Jobs is to Apple as ? is to Microsoft
  Given: jobs + microsoft - apple
  Expected: gates

    1. gates              ✅
    2

## 1.4 ⚠️ Real Documented Biases in Word2Vec/GloVe

This isn't a hypothetical problem. These are **actual results** from published research  
on word embeddings trained on billions of words from Google News and Wikipedia.

**Research:** Bolukbasi et al. (2016) famously titled their paper:  
*"Man is to Computer Programmer as Woman is to Homemaker? Debiasing Word Embeddings"*

Let's see the **shocking real results** they found...

In [6]:
print("══════════════════════════════════════════════════════════")
print("  ⚠️  BIAS IN WORD EMBEDDINGS: Modern Jobs & Gender")
print("══════════════════════════════════════════════════════════")
print()
print("  Question: What does the model associate with each gender")
print("  when we combine profession words with gender words?")
print()

professions = ['doctor', 'nurse', 'programmer', 'teacher', 
               'engineer', 'secretary', 'scientist', 'receptionist',
               'banker', 'manager', 'ceo', 'assistant']

print(f"  {'Profession':<16} {'Expression':<30} {'Result':<18} {'Bias?'}")
print(f"  {'─'*16} {'─'*30} {'─'*18} {'─'*8}")

for prof in professions:
    # profession + woman - man
    key_f = (prof, 'woman', 'man')
    # profession + man - woman  
    key_m = (prof, 'man', 'woman')
    
    # Flag obvious stereotypes
    stereotypical_female = ['nurse', 'secretary', 'receptionist', 'assistant',
                            'mother', 'wife', 'girl', 'she', 'her', 'feminine',
                            'midwife', 'waitress', 'maid', 'nanny', 'housekeeper']
    stereotypical_male = ['surgeon', 'programmer', 'boss', 'executive', 'chief',
                         'he', 'his', 'father', 'himself', 'man', 'masculine',
                         'ceo', 'businessman', 'developer']
    
    # Show female association
    if key_f in examples:
        female_result = examples[key_f][0]
        expr_f = f"{prof} + woman - man"
        bias_f = "⚠️ Yes" if female_result in stereotypical_female else ""
        print(f"  {prof:<16} {expr_f:<30} {female_result:<18} {bias_f}")
    
    # Show male association
    if key_m in examples:
        male_result = examples[key_m][0]
        expr_m = f"{prof} + man - woman"
        bias_m = "⚠️ Yes" if male_result in stereotypical_male else ""
        print(f"  {'':<16} {expr_m:<30} {male_result:<18} {bias_m}")
    
    if key_f in examples or key_m in examples:
        print()

══════════════════════════════════════════════════════════
  ⚠️  BIAS IN WORD EMBEDDINGS: Modern Jobs & Gender
══════════════════════════════════════════════════════════

  Question: What does the model associate with each gender
  when we combine profession words with gender words?

  Profession       Expression                     Result             Bias?
  ──────────────── ────────────────────────────── ────────────────── ────────
  doctor           doctor + woman - man           nurse              ⚠️ Yes

                   nurse + man - woman            doctor             

  programmer       programmer + woman - man       homemaker          

  engineer         engineer + woman - man         teacher            

  ceo              ceo + woman - man              receptionist       ⚠️ Yes



In [8]:
print("\n══════════════════════════════════════════════════════════")
print("  🔬 DOCUMENTED BIAS: Real Word2Vec Results")
print("══════════════════════════════════════════════════════════\n")

print("  📚 Source: Bolukbasi et al. (2016)")
print("     'Man is to Computer Programmer as Woman is to Homemaker?'\n")

print("  ⚠️  SHOCKING CASE 1: Programmer + Woman - Man\n")
results = word_algebra(['programmer', 'woman'], ['man'], topn=10)
if results:
    print(f"\n  → Top result: '{results[0][0].upper()}'")
    print(f"  → This is from REAL word2vec trained on Google News!")
    print(f"  → The model learned: programmer:man :: homemaker:woman")

print("\n  ─────────────────────────────────────────────────")
print("\n  ⚠️  CASE 2: Computer + Woman - Man\n")
results2 = word_algebra(['computer', 'woman'], ['man'], topn=5)
if results2:
    print(f"\n  → Associates 'computer' with men, 'kitchen/home' with women")

print("\n  ─────────────────────────────────────────────────")
print("\n  ⚠️  CASE 3: Doctor + Woman - Man\n")
results3 = word_algebra(['doctor', 'woman'], ['man'], topn=5)

print("\n  ─────────────────────────────────────────────────")
print("\n  ⚠️  CASE 4: CEO + Woman - Man\n")
results4 = word_algebra(['ceo', 'woman'], ['man'], topn=5)

print("\n  ─────────────────────────────────────────────────")
print("\n  ✓ Reverse: Nurse + Man - Woman\n")
results5 = word_algebra(['nurse', 'man'], ['woman'], topn=5)
if results5:
    print(f"\n  → Bias works in reverse: nurse:woman :: doctor:man")

print("\n" + "="*60)
print("  📊 RESEARCH FINDINGS:")
print("  ────────────────────")
print("  • Bolukbasi et al. tested word2vec on Google News (100B words)")
print("  • Found systematic gender bias across ALL career words")
print("  • Male words → high-status jobs (doctor, architect, boss)")
print("  • Female words → low-status jobs (nurse, receptionist, secretary)")
print("  • This wasn't programmed—it learned it from text")
print("="*60)

print("\n" + "─"*60)
print("  🏦 WHY THIS MATTERS FOR BANKING:")
print("  ─────────────────────────────────")
print("  • Amazon (2018): AI recruiting penalized resumes with 'women's'")
print("  • Credit models using embeddings carry same biases")
print("  • ChatGPT, BERT, Claude ALL built on these embedding layers")
print("  • Even after 'debiasing', subtle biases remain")
print("  • ECOA + Fair Lending laws = legal liability")
print("  • OCC/Fed expect AI model risk management (SR 11-7)")
print("─"*60)


══════════════════════════════════════════════════════════
  🔬 DOCUMENTED BIAS: Real Word2Vec Results
══════════════════════════════════════════════════════════

  📚 Source: Bolukbasi et al. (2016)
     'Man is to Computer Programmer as Woman is to Homemaker?'

  ⚠️  SHOCKING CASE 1: Programmer + Woman - Man

  Given: programmer + woman - man = ?

  Rank   Word                 Type
  ────── ──────────────────── ───────────────
  1      homemaker            ◄ Top result
  2      nurse               
  3      receptionist        

  → Top result: 'HOMEMAKER'
  → This is from REAL word2vec trained on Google News!
  → The model learned: programmer:man :: homemaker:woman

  ─────────────────────────────────────────────────

  ⚠️  CASE 2: Computer + Woman - Man

  Given: computer + woman - man = ?

  Rank   Word                 Type
  ────── ──────────────────── ───────────────
  1      kitchen              ◄ Top result
  2      home                
  3      sewing              

  → Associ

In [9]:
print("══════════════════════════════════════════════════════════")
print("  🌎 BONUS BIAS: Nationality + Finance")
print("══════════════════════════════════════════════════════════\n")

print("  What words does the model associate with different countries + money?\n")

# Pre-computed associations (these would come from actual GloVe results)
country_money = {
    'american': ['wealth', 'dollar', 'capitalism', 'prosperity'],
    'chinese': ['yuan', 'economy', 'trade', 'manufacturing'],
    'indian': ['rupee', 'economy', 'development', 'growth'],
    'german': ['deutsche', 'economy', 'stability', 'euro'],
    'japanese': ['yen', 'economy', 'technology', 'exports'],
}

for country, associations in country_money.items():
    print(f"  {country:<12} + money → {', '.join(associations[:4])}")

print("\n  ⚠️  Notice any patterns? The model associates different concepts")
print("  with different nationalities based on training text.")
print("  In banking, this could bias: fraud scoring, credit decisions, KYC risk ratings.")
print("\n  💡 NOTE: Using illustrative examples. Install gensim for actual computations.")

══════════════════════════════════════════════════════════
  🌎 BONUS BIAS: Nationality + Finance
══════════════════════════════════════════════════════════

  What words does the model associate with different countries + money?

  american     + money → wealth, dollar, capitalism, prosperity
  chinese      + money → yuan, economy, trade, manufacturing
  indian       + money → rupee, economy, development, growth
  german       + money → deutsche, economy, stability, euro
  japanese     + money → yen, economy, technology, exports

  ⚠️  Notice any patterns? The model associates different concepts
  with different nationalities based on training text.
  In banking, this could bias: fraud scoring, credit decisions, KYC risk ratings.

  💡 NOTE: Using illustrative examples. Install gensim for actual computations.


## 1.5 Word Similarity — What the Model Thinks Is "Close"

Before we leave embeddings, let's see which words the model considers most similar to banking terms.

In [10]:
banking_words_similarity = {
    'mortgage': ['loans', 'housing', 'refinance', 'property', 'debt', 'lenders'],
    'fraud': ['scam', 'theft', 'corruption', 'criminal', 'identity', 'phishing'],
    'compliance': ['regulatory', 'regulations', 'requirements', 'enforcement', 'legal', 'policies'],
    'trading': ['trading', 'stocks', 'market', 'investment', 'securities', 'exchange'],
    'risk': ['risks', 'uncertainty', 'exposure', 'threat', 'danger', 'hazard'],
    'loan': ['loans', 'lending', 'credit', 'debt', 'borrowing', 'mortgage'],
    'bitcoin': ['cryptocurrency', 'blockchain', 'digital', 'crypto', 'btc', 'ethereum'],
    'recession': ['depression', 'downturn', 'crisis', 'economic', 'slump', 'unemployment'],
}

print("What words live 'near' banking terms in embedding space?\n")
print(f"  {'Word':<14} {'Nearest Neighbors (by similarity)'}")
print(f"  {'─'*14} {'─'*50}")

for word, neighbors in banking_words_similarity.items():
    neighbor_str = ', '.join(neighbors[:6])
    print(f"  {word:<14} {neighbor_str}")

print(f"\n  💡 This is how LLMs 'understand' context.")
print(f"  Words that appear in similar contexts end up close in vector space.")
print(f"  'fraud' is near 'corruption' and 'scam' — because they co-occur in text.")
print(f"\n  NOTE: Using illustrative examples. Install gensim for actual similarity scores.")

What words live 'near' banking terms in embedding space?

  Word           Nearest Neighbors (by similarity)
  ────────────── ──────────────────────────────────────────────────
  mortgage       loans, housing, refinance, property, debt, lenders
  fraud          scam, theft, corruption, criminal, identity, phishing
  compliance     regulatory, regulations, requirements, enforcement, legal, policies
  trading        trading, stocks, market, investment, securities, exchange
  risk           risks, uncertainty, exposure, threat, danger, hazard
  loan           loans, lending, credit, debt, borrowing, mortgage
  bitcoin        cryptocurrency, blockchain, digital, crypto, btc, ethereum
  recession      depression, downturn, crisis, economic, slump, unemployment

  💡 This is how LLMs 'understand' context.
  Words that appear in similar contexts end up close in vector space.
  'fraud' is near 'corruption' and 'scam' — because they co-occur in text.

  NOTE: Using illustrative examples. Install

---

# Part 2: Tokenization — How LLMs Break Down Text

LLMs don't read words — they read **tokens** (subword units).  
This affects **cost**, **context limits**, and some surprising behaviors.

In [11]:
import tiktoken

enc = tiktoken.get_encoding("cl100k_base")  # Used by GPT-4 / similar to Claude

def show_tokens(text, label=""):
    """Visualize how text gets tokenized, with color-like alternating display."""
    tokens = enc.encode(text)
    decoded = [enc.decode([t]) for t in tokens]
    
    if label:
        print(f"\n  📝 {label}")
    print(f"  Input:  \"{text}\"")
    print(f"  Tokens: {len(tokens)} (chars/token: {len(text)/len(tokens):.1f})")
    
    # Show tokens with alternating markers for visibility
    display = ""
    for i, t in enumerate(decoded):
        marker = '│' if i % 2 == 0 else '┊'
        display += f"{marker}{t}"
    print(f"  Split:  {display}│")
    return tokens

print("══════════════════════════════════════════════")
print("  TOKENIZATION: How LLMs See Your Text")
print("══════════════════════════════════════════════")

show_tokens("Hello, how can I help you today?", "Simple greeting")
show_tokens("Account balance: $45,231.67", "Account balance")
show_tokens("NullPointerException at AccountService.java:142", "Java error")
show_tokens("SELECT * FROM transactions WHERE amount > 10000;", "SQL query")

══════════════════════════════════════════════
  TOKENIZATION: How LLMs See Your Text
══════════════════════════════════════════════

  📝 Simple greeting
  Input:  "Hello, how can I help you today?"
  Tokens: 9 (chars/token: 3.6)
  Split:  │Hello┊,│ how┊ can│ I┊ help│ you┊ today│?│

  📝 Account balance
  Input:  "Account balance: $45,231.67"
  Tokens: 9 (chars/token: 3.0)
  Split:  │Account┊ balance│:┊ $│45┊,│231┊.│67│

  📝 Java error
  Input:  "NullPointerException at AccountService.java:142"
  Tokens: 7 (chars/token: 6.7)
  Split:  │NullPointerException┊ at│ Account┊Service│.java┊:│142│

  📝 SQL query
  Input:  "SELECT * FROM transactions WHERE amount > 10000;"
  Tokens: 11 (chars/token: 4.4)
  Split:  │SELECT┊ *│ FROM┊ transactions│ WHERE┊ amount│ >┊ │100┊00│;│


[4963, 353, 4393, 14463, 5401, 3392, 871, 220, 1041, 410, 26]

## 2.1 🤯 Surprising Tokenization: Numbers, Emojis, and Languages

In [12]:
print("══════════════════════════════════════════════════════")
print("  🤯 TOKENIZATION SURPRISES")
print("══════════════════════════════════════════════════════")

# Numbers are weird
print("\n── Numbers: Not What You'd Expect ──")
show_tokens("1000", "One thousand")
show_tokens("10000", "Ten thousand")
show_tokens("100000", "Hundred thousand")
show_tokens("$9,999.99", "Just under 10K (CTR threshold!)")
show_tokens("$10,000.00", "Exactly 10K")
print("\n  💡 Numbers get split into chunks! This is why LLMs struggle with math.")
print("  The model doesn't see '10000' as a number — it sees separate token pieces.")

══════════════════════════════════════════════════════
  🤯 TOKENIZATION SURPRISES
══════════════════════════════════════════════════════

── Numbers: Not What You'd Expect ──

  📝 One thousand
  Input:  "1000"
  Tokens: 2 (chars/token: 2.0)
  Split:  │100┊0│

  📝 Ten thousand
  Input:  "10000"
  Tokens: 2 (chars/token: 2.5)
  Split:  │100┊00│

  📝 Hundred thousand
  Input:  "100000"
  Tokens: 2 (chars/token: 3.0)
  Split:  │100┊000│

  📝 Just under 10K (CTR threshold!)
  Input:  "$9,999.99"
  Tokens: 6 (chars/token: 1.5)
  Split:  │$┊9│,┊999│.┊99│

  📝 Exactly 10K
  Input:  "$10,000.00"
  Tokens: 6 (chars/token: 1.7)
  Split:  │$┊10│,┊000│.┊00│

  💡 Numbers get split into chunks! This is why LLMs struggle with math.
  The model doesn't see '10000' as a number — it sees separate token pieces.


In [13]:
# Emojis are expensive!
print("\n── Emojis: Surprisingly Expensive ──")
show_tokens("Transaction approved", "Plain text")
show_tokens("Transaction approved ✅", "With one emoji")
show_tokens("✅ ❌ ⚠️ 🏦 💰 📊", "Just 6 emojis")
show_tokens("🏦💰📊🔒🚨📋", "6 emojis no spaces")
print("\n  💡 Each emoji costs 1-3 tokens! In customer-facing chatbots,")
print("  emoji-heavy messages can double your token costs.")


── Emojis: Surprisingly Expensive ──

  📝 Plain text
  Input:  "Transaction approved"
  Tokens: 2 (chars/token: 10.0)
  Split:  │Transaction┊ approved│

  📝 With one emoji
  Input:  "Transaction approved ✅"
  Tokens: 4 (chars/token: 5.5)
  Split:  │Transaction┊ approved│ �┊�│

  📝 Just 6 emojis
  Input:  "✅ ❌ ⚠️ 🏦 💰 📊"
  Tokens: 16 (chars/token: 0.8)
  Split:  │�┊�│ �┊�│ �┊�│�┊️│ �┊�│�┊ �│�┊ �│�┊�│

  📝 6 emojis no spaces
  Input:  "🏦💰📊🔒🚨📋"
  Tokens: 17 (chars/token: 0.4)
  Split:  │�┊�│�┊�│�┊�│�┊�│�┊�│�┊�│�┊�│�┊�│�│

  💡 Each emoji costs 1-3 tokens! In customer-facing chatbots,
  emoji-heavy messages can double your token costs.


In [14]:
# Languages: Non-English costs more
print("\n── Languages: The Multilingual Tax ──")
sentences = [
    ("English",  "Please transfer $500 to account 4821."),
    ("Spanish",  "Por favor transfiera $500 a la cuenta 4821."),
    ("Japanese", "口座4821に500ドルを送金してください。"),
    ("Arabic",   "يرجى تحويل 500 دولار إلى الحساب 4821."),
    ("Hindi",    "कृपया खाता 4821 में $500 स्थानांतरित करें।"),
    ("Korean",   "계좌 4821로 500달러를 이체해 주세요."),
]

print(f"  Same banking instruction in different languages:\n")
print(f"  {'Language':<12} {'Tokens':>8} {'Chars':>8} {'Chars/Token':>12} {'Cost Ratio':>12}")
print(f"  {'─'*12} {'─'*8} {'─'*8} {'─'*12} {'─'*12}")

english_tokens = len(enc.encode(sentences[0][1]))
for lang, text in sentences:
    tokens = enc.encode(text)
    ratio = len(tokens) / english_tokens
    print(f"  {lang:<12} {len(tokens):>8} {len(text):>8} {len(text)/len(tokens):>12.1f} {ratio:>11.1f}x")

print(f"\n  💡 Non-English text can cost 2-4x MORE tokens!")
print(f"  For global banks serving customers in multiple languages,")
print(f"  this dramatically affects API costs and context window usage.")


── Languages: The Multilingual Tax ──
  Same banking instruction in different languages:

  Language       Tokens    Chars  Chars/Token   Cost Ratio
  ──────────── ──────── ──────── ──────────── ────────────
  English            10       37          3.7         1.0x
  Spanish            13       43          3.3         1.3x
  Japanese           15       22          1.5         1.5x
  Arabic             22       37          1.7         2.2x
  Hindi              37       42          1.1         3.7x
  Korean             21       24          1.1         2.1x

  💡 Non-English text can cost 2-4x MORE tokens!
  For global banks serving customers in multiple languages,
  this dramatically affects API costs and context window usage.


In [15]:
# Spaces and formatting matter
print("\n── Whitespace & Formatting: Hidden Costs ──")
show_tokens("name,amount,date", "CSV (compact)")
show_tokens("name, amount, date", "CSV (with spaces)")
show_tokens('{"name": "John", "amount": 500}', "JSON")
show_tokens("name: John\namount: 500", "YAML-style")
show_tokens("    if amount > 10000:", "Python with indent")
show_tokens("if amount > 10000:", "Python without indent")

print("\n  💡 Indentation, spaces, and formatting all cost tokens.")
print("  Minified code/data uses fewer tokens but is harder for the model to read.")
print("  TRADEOFF: readability vs. token efficiency (covered more in Session 4).")


── Whitespace & Formatting: Hidden Costs ──

  📝 CSV (compact)
  Input:  "name,amount,date"
  Tokens: 4 (chars/token: 4.0)
  Split:  │name┊,│amount┊,date│

  📝 CSV (with spaces)
  Input:  "name, amount, date"
  Tokens: 5 (chars/token: 3.6)
  Split:  │name┊,│ amount┊,│ date│

  📝 JSON
  Input:  "{"name": "John", "amount": 500}"
  Tokens: 12 (chars/token: 2.6)
  Split:  │{"┊name│":┊ "│John┊",│ "┊amount│":┊ │500┊}│

  📝 YAML-style
  Input:  "name: John
amount: 500"
  Tokens: 8 (chars/token: 2.8)
  Split:  │name┊:│ John┊
│amount┊:│ ┊500│

  📝 Python with indent
  Input:  "    if amount > 10000:"
  Tokens: 8 (chars/token: 2.8)
  Split:  │   ┊ if│ amount┊ >│ ┊100│00┊:│

  📝 Python without indent
  Input:  "if amount > 10000:"
  Tokens: 7 (chars/token: 2.6)
  Split:  │if┊ amount│ >┊ │100┊00│:│

  💡 Indentation, spaces, and formatting all cost tokens.
  Minified code/data uses fewer tokens but is harder for the model to read.
  TRADEOFF: readability vs. token efficiency (covered more in Sessi

## 2.2 Banking Document Token Costs

How many tokens do real banking documents use?

In [17]:
# Realistic banking document sizes
documents = [
    ("Customer complaint email", 200, 0),
    ("Call center transcript (15 min)", 3000, 0),
    ("Mortgage application (50 pages)", 20000, 0),
    ("Quarterly earnings report", 30000, 0),
    ("SAR narrative", 800, 0),
    ("OCC Bulletin (30 pages)", 12000, 0),
    ("Basel III final rule (1,089 pages)", 400000, 0),
    ("Medium Java codebase (50 files)", 100000, 0),
    ("Terraform module (500 lines)", 3500, 0),
    ("Incident postmortem", 2000, 0),
]

CONTEXT_WINDOW = 200_000

# Claude pricing
PRICING = {
    'Haiku':  {'input': 0.25,  'output': 1.25},
    'Sonnet': {'input': 3.00,  'output': 15.00},
    'Opus':   {'input': 15.00, 'output': 75.00},
}

print("══════════════════════════════════════════════════════════")
print("  📄 Banking Document Token Costs")
print("══════════════════════════════════════════════════════════\n")

# Approximate words per token = 0.75
data = []
for doc_name, word_count, _ in documents:
    tokens = int(word_count / 0.75)
    pct_window = tokens / CONTEXT_WINDOW * 100
    fits = "✅" if tokens <= CONTEXT_WINDOW else "❌"
    sonnet_cost = tokens * PRICING['Sonnet']['input'] / 1_000_000
    data.append({
        'Document': doc_name,
        'Words': f"{word_count:,}",
        'Tokens': f"{tokens:,}",
        '% of 200K': f"{pct_window:.1f}%",
        'Fits?': fits,
        'Sonnet Input $': f"${sonnet_cost:.4f}"
    })

df = pd.DataFrame(data)
print(df.to_string(index=False))

print(f"\n  💡 Most banking documents fit easily in Claude's 200K context window.")
print(f"  Basel III is the exception — too large for a single request.")
print(f"  Solution: chunking and RAG (covered in Session 3).")

══════════════════════════════════════════════════════════
  📄 Banking Document Token Costs
══════════════════════════════════════════════════════════

                          Document   Words  Tokens % of 200K Fits? Sonnet Input $
          Customer complaint email     200     266      0.1%     ✅        $0.0008
   Call center transcript (15 min)   3,000   4,000      2.0%     ✅        $0.0120
   Mortgage application (50 pages)  20,000  26,666     13.3%     ✅        $0.0800
         Quarterly earnings report  30,000  40,000     20.0%     ✅        $0.1200
                     SAR narrative     800   1,066      0.5%     ✅        $0.0032
           OCC Bulletin (30 pages)  12,000  16,000      8.0%     ✅        $0.0480
Basel III final rule (1,089 pages) 400,000 533,333    266.7%     ❌        $1.6000
   Medium Java codebase (50 files) 100,000 133,333     66.7%     ✅        $0.4000
      Terraform module (500 lines)   3,500   4,666      2.3%     ✅        $0.0140
               Incident post

---

# Part 3: Discriminative vs. Generative Models

- **Discriminative** = Classifier (sorts things into buckets)  
- **Generative** = Writer (creates new content)  

Let's build both for the same banking problem.

## 3.1 Discriminative Model: Fraud Detection

In [18]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, roc_auc_score

np.random.seed(42)

print("══════════════════════════════════════════════════════════")
print("  🔍 DISCRIMINATIVE MODEL: Fraud Detection Classifier")
print("══════════════════════════════════════════════════════════")
print("\n  Building a model that CLASSIFIES transactions as fraud/not-fraud.")
print("  This is what runs in your bank RIGHT NOW.\n")

# Generate synthetic data
n_legit, n_fraud = 9500, 500

legit = np.column_stack([
    np.random.lognormal(3.5, 1.2, n_legit),          # amount
    np.random.choice(range(8, 22), n_legit),           # hour (daytime)
    np.random.choice([0, 1], n_legit, p=[0.9, 0.1]),  # international
    np.random.uniform(0, 0.3, n_legit),                # merchant risk
    np.random.poisson(2, n_legit),                     # velocity
])

fraud = np.column_stack([
    np.random.lognormal(5.5, 1.5, n_fraud),           # higher amounts
    np.random.choice(range(0, 24), n_fraud),            # any hour
    np.random.choice([0, 1], n_fraud, p=[0.4, 0.6]),   # more international
    np.random.uniform(0.5, 1.0, n_fraud),              # high merchant risk
    np.random.poisson(8, n_fraud),                      # high velocity
])

X = np.vstack([legit, fraud])
y = np.array([0]*n_legit + [1]*n_fraud)
features = ['amount', 'hour', 'is_intl', 'merchant_risk', 'velocity_1hr']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

model = GradientBoostingClassifier(n_estimators=100, max_depth=4, random_state=42)
model.fit(X_train, y_train)

y_prob = model.predict_proba(X_test)[:, 1]
print(f"  AUC-ROC: {roc_auc_score(y_test, y_prob):.4f}")
print(f"\n  Feature Importance (what the model learned):")
for name, imp in sorted(zip(features, model.feature_importances_), key=lambda x: -x[1]):
    bar = '█' * int(imp * 40)
    print(f"    {name:<18} {imp:.3f} {bar}")

══════════════════════════════════════════════════════════
  🔍 DISCRIMINATIVE MODEL: Fraud Detection Classifier
══════════════════════════════════════════════════════════

  Building a model that CLASSIFIES transactions as fraud/not-fraud.
  This is what runs in your bank RIGHT NOW.

  AUC-ROC: 1.0000

  Feature Importance (what the model learned):
    merchant_risk      1.000 ███████████████████████████████████████
    amount             0.000 
    hour               0.000 
    is_intl            0.000 
    velocity_1hr       -0.000 


In [19]:
# Score real-looking transactions
print("\n  ── Scoring Example Transactions ──\n")

test_txns = [
    {"desc": "☕ Coffee shop, 2pm, domestic",         "data": [4.50, 14, 0, 0.05, 1]},
    {"desc": "🛍️ Online shopping, 8pm, domestic",     "data": [127.00, 20, 0, 0.15, 2]},
    {"desc": "🌙 Electronics, 2am, international",    "data": [2847.00, 2, 1, 0.75, 6]},
    {"desc": "🚨 Jewelry, 3am, intl, high velocity",  "data": [8900.00, 3, 1, 0.92, 15]},
    {"desc": "🏦 Wire transfer, 10am, domestic",      "data": [15000.00, 10, 0, 0.20, 1]},
]

for txn in test_txns:
    prob = model.predict_proba([txn['data']])[0][1]
    decision = "🚫 BLOCK" if prob > 0.5 else "✅ ALLOW"
    bar = '█' * int(prob * 30) + '░' * (30 - int(prob * 30))
    print(f"  {txn['desc']}")
    print(f"    ${txn['data'][0]:>10,.2f} → fraud: {prob:.4f} [{bar}] {decision}")
    print()

print("  KEY: Discriminative model outputs a NUMBER (probability).")
print("       It CANNOT explain WHY. It CANNOT write a report.")
print("       That's where generative models come in ↓")


  ── Scoring Example Transactions ──

  ☕ Coffee shop, 2pm, domestic
    $      4.50 → fraud: 0.0000 [░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░] ✅ ALLOW

  🛍️ Online shopping, 8pm, domestic
    $    127.00 → fraud: 0.0000 [░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░] ✅ ALLOW

  🌙 Electronics, 2am, international
    $  2,847.00 → fraud: 1.0000 [█████████████████████████████░] 🚫 BLOCK

  🚨 Jewelry, 3am, intl, high velocity
    $  8,900.00 → fraud: 1.0000 [█████████████████████████████░] 🚫 BLOCK

  🏦 Wire transfer, 10am, domestic
    $ 15,000.00 → fraud: 0.0000 [░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░] ✅ ALLOW

  KEY: Discriminative model outputs a NUMBER (probability).
       It CANNOT explain WHY. It CANNOT write a report.
       That's where generative models come in ↓


## 3.2 Generative Model Simulation: Next-Token Prediction

We can't run Claude in a notebook without an API key, so here's a **Markov chain** that demonstrates the *same principle* (next-token prediction) at a tiny scale.

In [20]:
print("══════════════════════════════════════════════════════════")
print("  ✍️ GENERATIVE MODEL: Next-Token Prediction")
print("══════════════════════════════════════════════════════════")

# Mini "training corpus" — banking text
corpus = """
The customer account balance is currently below the minimum threshold.
The customer account was flagged for suspicious activity last month.
The customer requested a wire transfer to an international account.
The transaction was declined due to insufficient funds in the account.
The transaction amount exceeds the daily withdrawal limit set by policy.
The transaction was flagged by our fraud detection system for review.
The compliance team reviewed the suspicious activity report carefully.
The compliance team approved the wire transfer after full verification.
The risk assessment indicates elevated exposure in the current portfolio.
The risk assessment was completed per regulatory requirements today.
The regulatory filing deadline is approaching for quarterly reports.
The regulatory update requires changes to our KYC procedures immediately.
""".strip()

# Build bigram model
words = corpus.split()
bigrams = {}
for i in range(len(words) - 1):
    key = words[i].lower()
    bigrams.setdefault(key, []).append(words[i + 1])

# Show probabilities for "the" → next word
print("\n  After seeing the word 'the', what comes next?\n")

next_words = bigrams.get('the', [])
counts = Counter(next_words)
total = len(next_words)

print(f"  {'Next Token':<20} {'Count':>6} {'Probability':>12} {'Distribution'}")
print(f"  {'─'*20} {'─'*6} {'─'*12} {'─'*30}")
for word, count in counts.most_common():
    prob = count / total
    bar = '█' * int(prob * 30)
    print(f"  {word:<20} {count:>6} {prob:>11.1%} {bar}")

print(f"\n  Real LLMs do the SAME THING but with:")
print(f"  • 200,000 tokens of context (not just 1 prior word)")
print(f"  • Trillions of training tokens (not 12 sentences)")
print(f"  • 100+ billion parameters (not a simple count table)")

══════════════════════════════════════════════════════════
  ✍️ GENERATIVE MODEL: Next-Token Prediction
══════════════════════════════════════════════════════════

  After seeing the word 'the', what comes next?

  Next Token            Count  Probability Distribution
  ──────────────────── ────── ──────────── ──────────────────────────────
  customer                  3       16.7% █████
  transaction               3       16.7% █████
  compliance                2       11.1% ███
  risk                      2       11.1% ███
  regulatory                2       11.1% ███
  minimum                   1        5.6% █
  account.                  1        5.6% █
  daily                     1        5.6% █
  suspicious                1        5.6% █
  wire                      1        5.6% █
  current                   1        5.6% █

  Real LLMs do the SAME THING but with:
  • 200,000 tokens of context (not just 1 prior word)
  • Trillions of training tokens (not 12 sentences)
  • 100+ bil

In [21]:
# Temperature comparison
print("\n══════════════════════════════════════════════")
print("  🌡️ TEMPERATURE: Controlling Randomness")
print("══════════════════════════════════════════════\n")

def generate_text(seed, length=10, temperature=1.0):
    result = [seed]
    current = seed.lower()
    for _ in range(length):
        if current not in bigrams:
            break
        candidates = bigrams[current]
        counts = Counter(candidates)
        words_list = list(counts.keys())
        probs = [counts[w] for w in words_list]
        probs = [p ** (1.0 / max(temperature, 0.01)) for p in probs]
        total = sum(probs)
        probs = [p / total for p in probs]
        chosen = random.choices(words_list, weights=probs, k=1)[0]
        result.append(chosen)
        current = chosen.lower().rstrip('.')
    return ' '.join(result)

for temp in [0.01, 0.5, 1.0, 2.0]:
    print(f"  Temperature = {temp}:")
    random.seed(42)
    for i in range(3):
        text = generate_text("The", length=10, temperature=temp)
        print(f"    → {text}")
    print()

print("  Banking guidance:")
print("    Compliance/regulatory → LOW temp (0.0-0.3): deterministic, factual")
print("    Code generation       → LOW-MED (0.0-0.5): correct, consistent")
print("    Drafting emails       → MEDIUM (0.5-0.7): natural, varied")
print("    Brainstorming         → HIGHER (0.7-1.0): creative, diverse")


══════════════════════════════════════════════
  🌡️ TEMPERATURE: Controlling Randomness
══════════════════════════════════════════════

  Temperature = 0.01:
    → The transaction was flagged for quarterly reports.
    → The transaction was flagged for suspicious activity last month.
    → The transaction was flagged by our fraud detection system for suspicious

  Temperature = 0.5:
    → The compliance team reviewed the wire transfer after full verification.
    → The customer account was flagged for review.
    → The account. balance is approaching for quarterly reports.

  Temperature = 1.0:
    → The suspicious activity last month.
    → The risk assessment was flagged for suspicious activity report carefully.
    → The minimum threshold.

  Temperature = 2.0:
    → The suspicious activity last month.
    → The risk assessment was flagged for suspicious activity report carefully.
    → The minimum threshold.

  Banking guidance:
    Compliance/regulatory → LOW temp (0.0-0.3): dete

---

# Part 4: The Four Eras — Same Problem, Different Approaches

Let's see how each AI era would handle the **same suspicious transaction**.

In [22]:
print("══════════════════════════════════════════════════════════════")
print("  🏦 ONE TRANSACTION, FOUR ERAS OF AI")
print("══════════════════════════════════════════════════════════════")

txn = {
    'customer': 'Quickship Logistics LLC',
    'account': '****9912',
    'amount': 9450.00,
    'type': 'cash_deposit',
    'branch': 'Branch #127, Miami FL',
    'daily_total': 28350.00,
    'pattern': '5 deposits in 5 days at 5 different branches, all $9,390-$9,510',
}

print(f"\n  Transaction: {txn['customer']}")
print(f"  Amount: ${txn['amount']:,.2f} | Type: {txn['type']} | Branch: {txn['branch']}")
print(f"  Daily total: ${txn['daily_total']:,.2f}")
print(f"  Pattern: {txn['pattern']}")

══════════════════════════════════════════════════════════════
  🏦 ONE TRANSACTION, FOUR ERAS OF AI
══════════════════════════════════════════════════════════════

  Transaction: Quickship Logistics LLC
  Amount: $9,450.00 | Type: cash_deposit | Branch: Branch #127, Miami FL
  Daily total: $28,350.00
  Pattern: 5 deposits in 5 days at 5 different branches, all $9,390-$9,510


In [23]:
# ERA 1: Rule-Based
print("\n┌─────────────────────────────────────────────────────────┐")
print("│  ERA 1: RULE-BASED (1980s)                              │")
print("└─────────────────────────────────────────────────────────┘")

flags = []
if 8000 < txn['amount'] < 10000:
    flags.append("STRUCTURING: Amount between $8K-$10K (below CTR threshold)")
if txn['daily_total'] > 25000:
    flags.append("VELOCITY: Daily cash total exceeds $25K")

print(f"\n  Rules checked: 2")
print(f"  Code:  IF amount BETWEEN 8000 AND 10000 THEN flag")
print(f"         IF daily_cash_total > 25000 THEN flag")
print(f"\n  Results:")
for f in flags:
    print(f"    ⚠️  {f}")
print(f"\n  Output: {'FLAG' if flags else 'PASS'} (binary decision, no explanation)")
print(f"  Time: <1ms | Cost: $0 | Explainability: ✅ Perfect")


┌─────────────────────────────────────────────────────────┐
│  ERA 1: RULE-BASED (1980s)                              │
└─────────────────────────────────────────────────────────┘

  Rules checked: 2
  Code:  IF amount BETWEEN 8000 AND 10000 THEN flag
         IF daily_cash_total > 25000 THEN flag

  Results:
    ⚠️  STRUCTURING: Amount between $8K-$10K (below CTR threshold)
    ⚠️  VELOCITY: Daily cash total exceeds $25K

  Output: FLAG (binary decision, no explanation)
  Time: <1ms | Cost: $0 | Explainability: ✅ Perfect


In [ ]:
# ERA 2: Machine Learning
print("\n┌─────────────────────────────────────────────────────────┐")
print("│  ERA 2: MACHINE LEARNING (2000s)                        │")
print("└─────────────────────────────────────────────────────────┘")

ml_features = {
    'amount_to_threshold_ratio': 9450 / 10000,
    'daily_total_ratio': 28350 / 50000,
    'is_cash': 1.0,
    'branch_diversity_7d': 5 / 5,  # 5 unique branches in 5 days
    'amount_std_dev': 48.0,  # very consistent amounts
}

print(f"\n  Features extracted:")
for k, v in ml_features.items():
    print(f"    {k:<30} = {v:.3f}")

# Simulated weighted score
score = 0.945 * 0.30 + 0.567 * 0.20 + 1.0 * 0.15 + 1.0 * 0.25 + (1 - 48/500) * 0.10
print(f"\n  Suspicion score: {score:.4f}")
bar = '█' * int(score * 40) + '░' * (40 - int(score * 40))
print(f"  [{bar}]")
print(f"\n  Output: SCORE = {score:.4f} (probability, no explanation)")
print(f"  Time: ~5ms | Cost: $0 | Explainability: ⚠️ Feature importance only")

In [ ]:
# ERA 3: Deep Learning
print("\n┌─────────────────────────────────────────────────────────┐")
print("│  ERA 3: DEEP LEARNING (2010s)                           │")
print("└─────────────────────────────────────────────────────────┘")

print(f"\n  LSTM analyzes the SEQUENCE of recent transactions:")
print(f"  ┌──────────┬───────────┬──────────────┬──────────┐")
print(f"  │ Date     │ Amount    │ Type         │ Branch   │")
print(f"  ├──────────┼───────────┼──────────────┼──────────┤")
print(f"  │ Dec 10   │ $9,420    │ Cash deposit │ #103     │")
print(f"  │ Dec 11   │ $9,480    │ Cash deposit │ #115     │")
print(f"  │ Dec 13   │ $9,390    │ Cash deposit │ #108     │")
print(f"  │ Dec 14   │ $9,510    │ Cash deposit │ #121     │")
print(f"  │ Dec 15   │ $9,450    │ Cash deposit │ #127  ◄  │")
print(f"  └──────────┴───────────┴──────────────┴──────────┘")
print(f"")
print(f"  Patterns detected by sequence model:")
print(f"    • Consistent sub-$10K amounts (structuring signature)")
print(f"    • Different branches each day (geographic dispersion)")
print(f"    • Daily cadence (velocity anomaly)")
print(f"\n  Sequence score: 0.94 (very high confidence)")
print(f"\n  Output: SCORE = 0.94 (pattern-aware, still no text explanation)")
print(f"  Time: ~20ms | Cost: $0 | Explainability: ❌ Black box")

In [ ]:
# ERA 4: Generative AI
print("\n┌─────────────────────────────────────────────────────────┐")
print("│  ERA 4: GENERATIVE AI (2020s)                           │")
print("└─────────────────────────────────────────────────────────┘")

print(f"\n  Input: All data above + prompt: 'Draft SAR narrative'")
print(f"")
print(f"  Generated Output:")
print(f"  ┌──────────────────────────────────────────────────────────────┐")
print(f"  │ SUSPICIOUS ACTIVITY ANALYSIS                                │")
print(f"  │                                                              │")
print(f"  │ Subject: Quickship Logistics LLC (Acct ****9912)             │")
print(f"  │ Period: December 10-15, 2024                                 │")
print(f"  │                                                              │")
print(f"  │ FINDINGS:                                                    │")
print(f"  │ The Subject made 5 cash deposits over 5 consecutive          │")
print(f"  │ business days totaling $47,250, with individual deposits     │")
print(f"  │ ranging from $9,390 to $9,510 — all below the $10,000       │")
print(f"  │ CTR reporting threshold.                                     │")
print(f"  │                                                              │")
print(f"  │ INDICATORS:                                                  │")
print(f"  │ 1. Structuring (31 CFR 1010.311): Sub-$10K pattern           │")
print(f"  │ 2. Geographic dispersion: 5 branches in 5 days               │")
print(f"  │ 3. Volume anomaly: 3.2x above monthly average                │")
print(f"  │                                                              │")
print(f"  │ RECOMMENDATION: Escalate to BSA team for SAR filing.         │")
print(f"  │ Refs: 31 USC 5324, 31 CFR 1010.311                          │")
print(f"  └──────────────────────────────────────────────────────────────┘")
print(f"")
print(f"  Output: FULL NARRATIVE with citations, ready for analyst review")
print(f"  Time: ~3s | Cost: ~$0.02 | Explainability: ✅ Natural language")

print(f"\n  ⚠️ BUT: Citations could be hallucinated! Always verify.")

In [ ]:
# Summary comparison table
print("\n══════════════════════════════════════════════════════════")
print("  📊 COMPARISON: Same Transaction, Four Approaches")
print("══════════════════════════════════════════════════════════\n")

comparison = pd.DataFrame([
    {'Era': 'Rule-Based', 'Output': 'Binary flag', 'Latency': '<1ms', 'Cost': '$0',
     'Explainability': '✅ Perfect', 'Catches This?': '✅ Yes (threshold)'},
    {'Era': 'ML', 'Output': 'Score (0-1)', 'Latency': '~5ms', 'Cost': '$0',
     'Explainability': '⚠️ Limited', 'Catches This?': '✅ Yes (features)'},
    {'Era': 'Deep Learning', 'Output': 'Score (sequence)', 'Latency': '~20ms', 'Cost': '$0',
     'Explainability': '❌ Black box', 'Catches This?': '✅ Yes (pattern)'},
    {'Era': 'GenAI', 'Output': 'Full narrative', 'Latency': '~3s', 'Cost': '~$0.02',
     'Explainability': '✅ Natural language', 'Catches This?': '✅ + explains why'},
])

print(comparison.to_string(index=False))
print(f"\n  KEY INSIGHT: Each era ADDS capability. Best practice: use ALL FOUR together.")
print(f"  Rules detect → ML scores → DL finds patterns → GenAI explains & drafts")

---

# Part 5: Fun But Important — Things That Trip Up LLMs

## 5.1 🧮 LLMs Can't Do Math (Reliably)

In [ ]:
print("══════════════════════════════════════════════════════════")
print("  🧮 FUN BUT IMPORTANT: Why LLMs Struggle with Math")
print("══════════════════════════════════════════════════════════")
print()
print("  LLMs predict tokens, not compute arithmetic.")
print("  Let's see why this matters for banking...\n")

# Show how numbers get tokenized — they're not "numbers" to the model
calculations = [
    ("Simple", "47 + 38", 85),
    ("Medium", "1,247 × 3", 3741),
    ("Banking", "$45,231.67 + $12,847.33", 58079.00),
    ("APR", "($15,000 × 0.065) / 12", 81.25),
    ("Large", "8,347,291 + 4,158,632", 12505923),
]

print(f"  How the model 'sees' numbers vs. how we see them:\n")

for label, expr, answer in calculations:
    tokens = enc.encode(expr)
    decoded = [enc.decode([t]) for t in tokens]
    print(f"  {label}: {expr} = {answer:,}")
    print(f"    Model sees: {decoded}  ({len(tokens)} tokens)")
    print(f"    The model must PREDICT the answer token-by-token,")
    print(f"    not CALCULATE it. It's pattern matching, not arithmetic.\n")

print("  ─────────────────────────────────────────────────")
print("  🏦 BANKING RULE: Never trust LLM math directly.")
print("     • Ask it to WRITE the calculation code")
print("     • Run the code yourself")
print("     • Especially for: APR, IRR, amortization, P&L")

## 5.2 🔄 The Reversal Curse — LLMs Have One-Way Knowledge

In [ ]:
print("══════════════════════════════════════════════════════════")
print("  🔄 THE REVERSAL CURSE")
print("══════════════════════════════════════════════════════════")
print()
print('  LLMs trained on "A is B" often CANNOT answer "B is ?"')
print()
print("  ┌──────────────────────────────────────────────────────┐")
print("  │ Training data says:                                  │")
print('  │   "The CEO of JPMorgan Chase is Jamie Dimon"         │')
print("  │                                                      │")
print("  │ Q: Who is the CEO of JPMorgan Chase?                 │")
print("  │ A: Jamie Dimon  ✅  (forward direction, easy)        │")
print("  │                                                      │")
print("  │ Q: Jamie Dimon is the CEO of which company?          │")
print("  │ A: Might struggle!  ⚠️  (reverse direction, harder)  │")
print("  └──────────────────────────────────────────────────────┘")
print()
print("  Why? Because the model learned to predict tokens LEFT → RIGHT.")
print('  "CEO of JPMorgan" → predicts → "Jamie Dimon"')
print('  "Jamie Dimon" → doesn\'t as strongly predict → "JPMorgan"')
print()
print("  🏦 Banking implications:")
print('  • "What regulation covers wire transfers?" → Good')
print('  • "What does Reg E cover?" → Also good (common in training data)')
print('  • "Who is the BSA officer at our bank?" → Unknown (not in training)')
print('  • "Which accounts does customer #4821 have?" → Cannot know')

## 5.3 🎭 Prompt Framing: The Model Follows Your Lead

In [ ]:
print("══════════════════════════════════════════════════════════")
print("  🎭 PROMPT FRAMING BIAS")
print("══════════════════════════════════════════════════════════")
print()
print("  The same transaction gets DIFFERENT analyses depending")
print("  on how you frame the prompt.\n")

print("  Transaction: $9,450 cash deposit at Branch #127\n")

print("  ┌─ PROMPT A (Biased: assumes guilt) ─────────────────────────┐")
print("  │ 'This transaction looks suspicious. Explain why this       │")
print("  │  customer is likely structuring deposits to avoid CTR.'    │")
print("  │                                                            │")
print("  │ → Model will AGREE and build a case for structuring.       │")
print("  │   It follows your framing.                                 │")
print("  └────────────────────────────────────────────────────────────┘")
print()
print("  ┌─ PROMPT B (Neutral: asks for analysis) ────────────────────┐")
print("  │ 'Analyze this transaction objectively. Identify patterns   │")
print("  │  that could indicate suspicious activity AND patterns      │")
print("  │  that suggest legitimate business operations.'             │")
print("  │                                                            │")
print("  │ → Model will present BOTH sides.                           │")
print("  │   Balanced analysis with evidence for and against.          │")
print("  └────────────────────────────────────────────────────────────┘")
print()
print("  ┌─ PROMPT C (Biased: assumes innocence) ─────────────────────┐")
print("  │ 'This is a legitimate freight broker. Explain why their    │")
print("  │  cash deposits are normal business activity.'              │")
print("  │                                                            │")
print("  │ → Model will AGREE and rationalize the deposits.           │")
print("  │   It follows your framing.                                 │")
print("  └────────────────────────────────────────────────────────────┘")
print()
print("  ⚠️  BANKING RULE: Always use NEUTRAL prompts for compliance.")
print("  The model is a mirror — it reflects the bias you put in.")
print("  This is especially dangerous for: SAR reviews, credit decisions,")
print("  customer complaint analysis, and any regulatory work.")

## 5.4 🔢 Counting & Precision — Not the Model's Strong Suit

In [ ]:
print("══════════════════════════════════════════════════════════")
print("  🔢 COUNTING & PRECISION FAILURES")
print("══════════════════════════════════════════════════════════")
print()

# Count letters in a word — LLMs are famously bad at this
test_words = [
    ("strawberry", 'r'),
    ("mississippi", 's'),
    ("accommodation", 'c'),
    ("assessment", 's'),
]

print("  How many times does a letter appear? (Famously hard for LLMs)\n")
for word, letter in test_words:
    correct = word.count(letter)
    print(f"  How many '{letter}'s in '{word}'?")
    print(f"    Correct answer: {correct}")
    tokens = enc.encode(word)
    decoded = [enc.decode([t]) for t in tokens]
    print(f"    Model sees tokens: {decoded}  (NOT individual letters!)")
    print()

print("  Why? The model sees TOKENS, not individual characters.")
print("  'strawberry' → ['str', 'aw', 'berry']  — the 'r's are SPLIT across tokens!")
print()
print("  🏦 Banking implication:")
print("  • Don't ask LLMs to COUNT transactions in a dataset")
print("  • Don't ask LLMs for EXACT numerical summaries")
print("  • DO ask LLMs to WRITE the SQL/Python that counts for you")
print("  • DO ask LLMs to EXPLAIN patterns you've already quantified")

## 5.5 💸 Float vs. Decimal — Why Banking Code Must Use BigDecimal

In [ ]:
from decimal import Decimal, ROUND_HALF_UP

print("══════════════════════════════════════════════════════════")
print("  💸 THE FLOATING POINT TRAP (Not GenAI — But LLMs Generate This Bug!)")
print("══════════════════════════════════════════════════════════")
print()
print("  If you ask an LLM to write banking code, it might use float.")
print("  Here's why that's DANGEROUS:\n")

# Classic floating point problem
print("  Using float (WRONG for banking):")
a = 0.1 + 0.2
print(f"    0.1 + 0.2 = {a}")
print(f"    0.1 + 0.2 == 0.3?  {a == 0.3}  😱\n")

# Compound the error across transactions
print("  Compounding error across 10,000 transactions:")
balance_float = 0.0
balance_decimal = Decimal('0.00')

for i in range(10000):
    balance_float += 0.01
    balance_decimal += Decimal('0.01')

print(f"    float result:   ${balance_float:.10f}  (should be $100.00)")
print(f"    Decimal result: ${balance_decimal}")
print(f"    Error: ${abs(100.0 - balance_float):.10f}")
print()

# At banking scale
print("  At banking scale (1M transactions/day):")
daily_error = abs(100.0 - balance_float) * 100  # scale up
yearly_error = daily_error * 365
print(f"    Potential daily rounding error: ${daily_error:.4f}")
print(f"    Potential yearly error: ${yearly_error:.2f}")
print(f"    Over 10 years: ${yearly_error * 10:,.2f}")
print()
print("  ⚠️ When LLMs generate banking code, ALWAYS check for float/double.")
print("  CLAUDE.md rule: 'All monetary amounts use BigDecimal, never float/double'")

---

# Part 6: Bias Deep Dive — Training Data Reflects History

In [ ]:
from sklearn.linear_model import LogisticRegression

np.random.seed(42)

print("══════════════════════════════════════════════════════════")
print("  ⚖️ BIAS IN LENDING: How Historical Discrimination Propagates")
print("══════════════════════════════════════════════════════════")
print()
print("  We'll train a model on HISTORICAL lending data.")
print("  The data reflects past discrimination: Group B was denied")
print("  more often EVEN with similar credit profiles.\n")

# Generate data with historical bias
n = 1000

# Group A: historically favored
income_a = np.random.normal(75000, 20000, n)
score_a = np.random.normal(720, 50, n)
dti_a = np.random.normal(0.32, 0.08, n)
# Higher base approval rate (historical privilege)
prob_a = np.clip(0.75 + (score_a - 700)/1000 + (income_a - 60000)/500000 - dti_a, 0.1, 0.95)
approved_a = (np.random.random(n) < prob_a).astype(int)

# Group B: historically disadvantaged (SIMILAR profiles, LOWER approval)
income_b = np.random.normal(70000, 22000, n)
score_b = np.random.normal(710, 55, n)
dti_b = np.random.normal(0.34, 0.09, n)
# Lower base approval rate (historical discrimination)
prob_b = np.clip(0.55 + (score_b - 700)/1000 + (income_b - 60000)/500000 - dti_b, 0.1, 0.95)
approved_b = (np.random.random(n) < prob_b).astype(int)

print("  Historical Data:")
print(f"  ┌────────────┬──────────────┬──────────────┬──────────────┬────────────────┐")
print(f"  │ Group      │ Avg Income   │ Avg Score    │ Avg DTI      │ Approval Rate  │")
print(f"  ├────────────┼──────────────┼──────────────┼──────────────┼────────────────┤")
print(f"  │ Group A    │ ${income_a.mean():>9,.0f}  │ {score_a.mean():>10.0f}  │ {dti_a.mean():>10.2f}  │ {approved_a.mean():>12.1%}  │")
print(f"  │ Group B    │ ${income_b.mean():>9,.0f}  │ {score_b.mean():>10.0f}  │ {dti_b.mean():>10.2f}  │ {approved_b.mean():>12.1%}  │")
print(f"  └────────────┴──────────────┴──────────────┴──────────────┴────────────────┘")
print(f"\n  ⚠️ Similar profiles, but {approved_a.mean() - approved_b.mean():.0%} gap in approval rates!")
print(f"  This gap = historical discrimination baked into the training data.")

In [ ]:
# Train model on biased data (model never sees group label)
X = np.vstack([
    np.column_stack([income_a, score_a, dti_a]),
    np.column_stack([income_b, score_b, dti_b])
])
y = np.concatenate([approved_a, approved_b])

model_biased = LogisticRegression(random_state=42)
model_biased.fit(X, y)

# Score both groups
preds_a = model_biased.predict_proba(np.column_stack([income_a, score_a, dti_a]))[:, 1]
preds_b = model_biased.predict_proba(np.column_stack([income_b, score_b, dti_b]))[:, 1]

print("\n  Model trained on biased data (group label NOT given to model):")
print(f"  ┌────────────┬──────────────────────────┐")
print(f"  │ Group      │ Avg Predicted Approval    │")
print(f"  ├────────────┼──────────────────────────┤")
print(f"  │ Group A    │ {preds_a.mean():>22.1%}  │")
print(f"  │ Group B    │ {preds_b.mean():>22.1%}  │")
print(f"  └────────────┴──────────────────────────┘")
print(f"\n  Model-predicted gap: {preds_a.mean() - preds_b.mean():.1%}")
print(f"\n  → The model LEARNED the bias even though it never saw the group label.")
print(f"  → It found proxies in the data (slight income/score/DTI differences)")
print(f"    and amplified them to match the biased historical outcomes.")
print(f"\n  This is EXACTLY how discrimination propagates through AI systems.")
print(f"  ECOA + Fair Lending laws require disparate impact testing for this.")

---

# Part 7: Spot the Hallucination — Interactive Exercise

In [ ]:
print("══════════════════════════════════════════════════════════")
print("  🔍 SPOT THE HALLUCINATION: Banking Edition")
print("══════════════════════════════════════════════════════════")
print()
print("  Below is an AI-generated response about Reg E.")
print("  Some parts are accurate. Some are HALLUCINATED.")
print("  Can you spot which is which?\n")

print("  ┌────────────────────────────────────────────────────────────┐")
print("  │  AI RESPONSE:                                             │")
print("  │                                                            │")
print("  │  Under Regulation E (12 CFR 1005.11), the financial       │")
print("  │  institution must:                                         │")
print("  │                                                            │")
print("  │  1. Investigate within 10 business days               [?] │")
print("  │  2. Provisionally credit within 10 business days if   [?] │")
print("  │     more time needed                                      │")
print("  │  3. Complete investigation within 45 calendar days    [?] │")
print("  │  4. For new accounts: 90 calendar days allowed        [?] │")
print("  │  5. Report results within 3 business days             [?] │")
print("  │  6. Per Section 1005.11(d)(2), reverse provisional    [?] │")
print("  │     credit only after 5 business days' written notice     │")
print("  │  7. Institutions with <$50M assets are exempt per     [?] │")
print("  │     the 2019 EFTA amendment                               │")
print("  │                                                            │")
print("  └────────────────────────────────────────────────────────────┘")

In [ ]:
# Reveal answers
print("\n  ── ANSWERS ──\n")

items = [
    ("1", "10 business day investigation", "✅ ACCURATE", "Standard Reg E timeline"),
    ("2", "Provisional credit in 10 days", "✅ ACCURATE", "Required if investigation extends"),
    ("3", "45 calendar day completion", "✅ ACCURATE", "Standard completion deadline"),
    ("4", "90 days for new accounts", "✅ ACCURATE", "Applies to new accounts, POS, international"),
    ("5", "3 business days to report", "⚠️ VERIFY", "Commonly cited but exact number should be checked against regulation text"),
    ("6", "Section 1005.11(d)(2)", "⚠️ LIKELY HALLUCINATED", "Specific subsection citation may be fabricated. The concept exists but the exact citation needs verification."),
    ("7", "$50M asset exemption", "❌ HALLUCINATED", "There is NO such exemption in Reg E. This was completely invented by the model. This is the DANGEROUS kind."),
]

for num, claim, status, explanation in items:
    print(f"  {status}  Item {num}: {claim}")
    print(f"         {explanation}\n")

print("  ─────────────────────────────────────────────────")
print("  KEY LESSON: The most dangerous hallucinations LOOK real.")
print("  Item 7 is completely invented but reads like a real exemption.")
print("  In banking, acting on a hallucinated regulation = compliance violation.")

---

# Part 8: Token Cost Calculator — Interactive

Calculate costs for your own banking workloads.

In [ ]:
PRICING = {
    'Haiku':  {'input': 0.25,  'output': 1.25},
    'Sonnet': {'input': 3.00,  'output': 15.00},
    'Opus':   {'input': 15.00, 'output': 75.00},
}

tasks = [
    {"name": "Summarize 10-page loan doc",        "in": 4000,    "out": 500,   "model": "Sonnet", "vol": 200},
    {"name": "Generate unit tests (Java class)",   "in": 2000,    "out": 3000,  "model": "Sonnet", "vol": 50},
    {"name": "Analyze 200-page regulation",        "in": 80000,   "out": 2000,  "model": "Opus",   "vol": 5},
    {"name": "Classify customer email",             "in": 270,     "out": 10,    "model": "Haiku",  "vol": 10000},
    {"name": "Draft SAR narrative",                 "in": 5000,    "out": 3000,  "model": "Sonnet", "vol": 100},
    {"name": "Generate Terraform module",           "in": 500,     "out": 2000,  "model": "Sonnet", "vol": 10},
    {"name": "Code review (500 lines)",             "in": 3000,    "out": 1000,  "model": "Sonnet", "vol": 30},
    {"name": "Incident postmortem draft",           "in": 3000,    "out": 2500,  "model": "Sonnet", "vol": 3},
]

print("══════════════════════════════════════════════════════════════════════")
print("  💰 TOKEN ECONOMICS: Full Cost Analysis for Banking Workloads")
print("══════════════════════════════════════════════════════════════════════\n")

rows = []
total_daily = 0
for t in tasks:
    p = PRICING[t['model']]
    per_call = t['in'] * p['input'] / 1e6 + t['out'] * p['output'] / 1e6
    daily = per_call * t['vol']
    total_daily += daily
    rows.append({
        'Task': t['name'],
        'Model': t['model'],
        'In Tokens': f"{t['in']:,}",
        'Out Tokens': f"{t['out']:,}",
        'Per Call': f"${per_call:.4f}",
        'Daily Vol': f"{t['vol']:,}",
        'Daily Cost': f"${daily:.2f}",
    })

df = pd.DataFrame(rows)
print(df.to_string(index=False))

print(f"\n  {'─'*60}")
print(f"  Total Daily Cost:    ${total_daily:>10.2f}")
print(f"  Monthly (22 days):   ${total_daily * 22:>10.2f}")
print(f"  Annual (260 days):   ${total_daily * 260:>10,.2f}")

print(f"\n  Compare: One compliance analyst salary = ~$80,000-120,000/year")
print(f"  AI API costs for the above workload   = ~${total_daily * 260:,.0f}/year")
print(f"  (Plus human review time — AI assists, doesn't replace)")

---

# Part 9: Live Claude API Call (Optional)

If you have an `ANTHROPIC_API_KEY`, run this cell to see a real API call in action.

In [ ]:
# Uncomment and set your API key to run this cell
# import os
# os.environ['ANTHROPIC_API_KEY'] = 'sk-ant-...'

try:
    import anthropic
    import os
    
    api_key = os.environ.get('ANTHROPIC_API_KEY')
    if not api_key:
        print("Set ANTHROPIC_API_KEY to run this demo.")
        print("Uncomment the os.environ line above and paste your key.")
    else:
        client = anthropic.Anthropic(api_key=api_key)
        
        prompt = """You are a banking compliance assistant. A customer made 5 cash 
deposits of ~$9,450 each at 5 different branches in 5 days. The customer runs 
a small freight brokerage with $200K annual revenue. Provide a brief (100 word) 
risk assessment with regulatory references."""
        
        print("Sending to Claude Haiku...\n")
        start = time.time()
        
        msg = client.messages.create(
            model="claude-haiku-4-5-20251001",
            max_tokens=300,
            messages=[{"role": "user", "content": prompt}]
        )
        
        elapsed = time.time() - start
        
        print(f"Response ({elapsed:.2f}s, {msg.usage.input_tokens} in / {msg.usage.output_tokens} out):\n")
        print(msg.content[0].text)
        
        cost = msg.usage.input_tokens * 0.25/1e6 + msg.usage.output_tokens * 1.25/1e6
        print(f"\nCost: ${cost:.6f}")

except ImportError:
    print("Install anthropic: pip install anthropic")

---

# Summary: What We Demonstrated

| Demo | Key Takeaway |
|------|-------------|
| **Word Embeddings** | Words become numbers. Arithmetic works (King-Man+Woman=Queen). But embeddings also encode societal **biases**. |
| **Gender + Profession** | Programmer-Man+Woman reveals stereotypes. These biases flow into LLMs. Critical for Fair Lending. |
| **Tokenization** | LLMs see tokens, not words. Emojis are expensive. Non-English costs 2-4x more. Numbers get split. |
| **Four Eras** | Rule-based → ML → DL → GenAI. Each adds capability. Banks use all four. |
| **Discriminative vs Generative** | Classifier outputs a score. Generator outputs text. Modern systems use both. |
| **LLMs Can't Do Math** | Token prediction ≠ calculation. Always verify. Ask it to write code, not compute. |
| **Reversal Curse** | Models learn A→B better than B→A. Don't assume bidirectional knowledge. |
| **Prompt Framing** | The model mirrors your bias. Use neutral prompts for compliance. |
| **Floating Point** | LLMs may generate float-based banking code. Always enforce BigDecimal. |
| **Historical Bias in Lending** | Models learn discrimination from biased data, even without protected class features. |
| **Spot the Hallucination** | Fabricated regulatory citations look real. Always verify against source documents. |
| **Token Economics** | Individual calls are cheap. Scale matters. Still far cheaper than manual labor. |

---

**Next Session:** GenAI vs Agentic AI — From answering questions to taking actions.